# Construindo e implantando soluções de aprendizado de máquina com Vertex AI: Challenge Lab

Este laboratório de desafios é recomendado para alunos que se inscreveram no [**Construindo e implantando soluções de aprendizado de máquina com o Vertex AI**](). Você receberá um cenário e um conjunto de tarefas. Em vez de seguir instruções passo a passo, você usará as habilidades aprendidas nos laboratórios da missão para descobrir como concluir as tarefas por conta própria! Um sistema de pontuação automatizado (mostrado na página de instruções do laboratório do Qwiklabs) fornecerá feedback sobre se você concluiu suas tarefas corretamente.

Ao participar de um laboratório de desafio, você não aprenderá os conceitos do Google Cloud. Para criar a solução para o desafio apresentado, use as habilidades aprendidas nos laboratórios da Quest da qual este laboratório de desafio faz parte. Espera-se que você amplie suas habilidades aprendidas e complete todos os comentários **`TODO:`** neste caderno.

Você está pronto para o desafio?

## Cenário

Você foi contratado recentemente como engenheiro de aprendizado de máquina em um site de resenhas de filmes de startups. Seu gerente encarregou você de criar um modelo de aprendizado de máquina para classificar o sentimento das resenhas de filmes do usuário como positivo ou negativo. Essas previsões serão usadas como uma entrada nos sistemas de classificação de filmes downstream e para exibir as principais críticas de apoio e críticas no aplicativo do site do filme. O desafio: seus requisitos de negócios são que você tenha apenas 6 semanas para produzir um modelo que atinja mais de 75% de precisão para melhorar uma solução bootstrap existente. Além disso, depois de fazer algumas análises exploratórias no data warehouse de sua startup, você descobriu que tem apenas um pequeno conjunto de dados de 50 mil revisões de texto para construir uma solução de melhor desempenho.

Para criar e implantar rapidamente um modelo de aprendizado de máquina de alto desempenho com dados limitados, você passará pelo treinamento e pela implantação de um classificador de sentimento personalizado do TensorFlow BERT para previsões on-line no [Vertex AI] do Google Cloud(https://cloud.google.com/vertex-ai) plataforma. A Vertex AI é a plataforma de desenvolvimento de aprendizado de máquina de última geração do Google Cloud, na qual você pode aproveitar os componentes pré-criados de ML mais recentes e o AutoML para melhorar significativamente sua produtividade de desenvolvimento, dimensionar seu fluxo de trabalho e tomar decisões com seus dados e acelerar o tempo de retorno.

![Vertex AI: Challenge Lab](./images/vertex-challenge-lab.png "Vertex Challenge Lab")

Primeiro, você irá progredir através de um fluxo de trabalho de experimentação típico onde você construirá seu modelo a partir de componentes BERT pré-treinados das camadas de classificação TF-Hub e `tf.keras` para treinar e avaliar seu modelo em um Vertex Notebook. Em seguida, você empacotará seu código de modelo em um contêiner do Docker para treinar no Vertex AI do Google Cloud. Por fim, você definirá e executará um Kubeflow Pipeline no Vertex Pipelines que treina e implanta seu modelo em um Vertex Endpoint que você consultará para previsões online.

## Objetivos de aprendizado

* Treine um modelo do TensorFlow localmente em um [**Vertex Notebook**](https://cloud.google.com/vertex-ai/docs/general/notebooks?hl=sv) hospedado.
* Coloque seu código de treinamento em contêiner com o [**Cloud Build**](https://cloud.google.com/build) e envie-o para o [**Google Cloud Artifact Registry**](https://cloud.google.com/artifact-registry).
* Defina um pipeline usando o [**Kubeflow Pipelines (KFP) V2 SDK**](https://www.kubeflow.org/docs/components/pipelines/sdk/v2/v2-compatibility) para treinar e implantar seu modelo em [**Vertex Pipelines**](https://cloud.google.com/vertex-ai/docs/pipelines).
* Consulte seu modelo em um [**Vertex Endpoint**](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) usando previsões on-line.

## Configurações

### Definição de Constantes

In [ ]:
# Adicione dependências de biblioteca instaladas à variável Python PATH.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

In [ ]:
# Recupere e defina as variáveis de ambiente PROJECT_ID e REGION.
# TODO: preencha PROJECT_ID.
PROJECT_ID = ""
REGION = "us-central1"

In [ ]:
# TODO: Crie um bucket globalmente exclusivo do Google Cloud Storage para armazenamento de artefatos.
GCS_BUCKET = ""

In [ ]:
!gsutil mb -l $REGION $GCS_BUCKET

### Import libraries

In [ ]:
import os
import shutil
import logging

# Bibliotecas de construção de modelos do TensorFlow.
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub

# Recrie o otimizador AdamW usado no documento BERT original.
from official.nlp import optimization  

# Bibliotecas para métricas de treinamento de modelos de dados e gráficos.
import pandas as pd
import matplotlib.pyplot as plt

# Importe o SDK do Vertex AI Python.
from google.cloud import aiplatform as vertexai

### Inicialize o SDK do Vertex AI Python

Inicialize o Vertex AI Python SDK com seu projeto, região e bucket do Google Cloud Storage do GCP.

In [ ]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

## Crie e treine seu modelo localmente em um Vertex Notebook

Observação: este laboratório adapta e estende o [tutorial de classificação de texto TensorFlow BERT](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) oficial para utilizar os serviços Vertex AI. Consulte o tutorial para obter cobertura adicional sobre o ajuste fino de modelos BERT usando o TensorFlow.

### Conjunto de dados do laboratório

Neste laboratório, você usará o [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment) que contém o texto de 50.000 resenhas de filmes do Internet Movie Database. Eles são divididos em 25.000 avaliações para treinamento e 25.000 avaliações para teste. Os conjuntos de treinamento e teste são balanceados, o que significa que contêm um número igual de avaliações positivas e negativas. O código de ingestão e processamento de dados foi fornecido para você abaixo:

### Import dataset

In [ ]:
DATA_URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
LOCAL_DATA_DIR = "."

In [ ]:
def download_data(data_url, local_data_dir):
    """Download dataset.
    Args:
      data_url(str): Source data URL path.
      local_data_dir(str): Local data download directory path.
    Returns:
      dataset_dir(str): Local unpacked data directory path.
    """
    if not os.path.exists(local_data_dir):
        os.makedirs(local_data_dir)
    
    dataset = tf.keras.utils.get_file(
      fname="aclImdb_v1.tar.gz",
      origin=data_url,
      untar=True,
      cache_dir=local_data_dir,
      cache_subdir="")
    
    dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")
    
    train_dir = os.path.join(dataset_dir, "train")
    
    # Remova as pastas não utilizadas para facilitar o carregamento dos dados.
    remove_dir = os.path.join(train_dir, "unsup")
    shutil.rmtree(remove_dir)
    
    return dataset_dir

In [ ]:
DATASET_DIR = download_data(data_url=DATA_URL, local_data_dir=LOCAL_DATA_DIR)

In [ ]:
# Crie um dicionário para adicionar de forma iterativa o pipeline de dados e os hiperparâmetros de treinamento do modelo.
HPARAMS = {
    # Defina uma semente de amostragem aleatória para evitar vazamento de dados em divisões de dados de arquivos.
    "seed": 42,
    # Número de exemplos de treinamento e inferência.
    "batch-size": 32
}

In [ ]:
def load_datasets(dataset_dir, hparams):
    """Load pre-split tf.datasets.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      raw_train_ds(tf.dataset): Train split dataset (20k examples).
      raw_val_ds(tf.dataset): Validation split dataset (5k examples).
      raw_test_ds(tf.dataset): Test split dataset (25k examples).
    """    

    raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'train'),
        batch_size=hparams['batch-size'],
        validation_split=0.2,
        subset='training',
        seed=hparams['seed'])    

    raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'train'),
        batch_size=hparams['batch-size'],
        validation_split=0.2,
        subset='validation',
        seed=hparams['seed'])

    raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'test'),
        batch_size=hparams['batch-size'])
    
    return raw_train_ds, raw_val_ds, raw_test_ds

In [ ]:
raw_train_ds, raw_val_ds, raw_test_ds = load_datasets(DATASET_DIR, HPARAMS)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
CLASS_NAMES = raw_train_ds.class_names

train_ds = raw_train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = raw_test_ds.prefetch(buffer_size=AUTOTUNE)

Let's print a few example reviews:

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review {i}: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({CLASS_NAMES[label]})')

### Escolha um modelo BERT pré-treinado para ajustar e obter maior precisão

[**Bidirectional Encoder Representations from Transformers (BERT)**](https://arxiv.org/abs/1810.04805v2) é um modelo de representação de texto baseado em transformador pré-treinado em grandes conjuntos de dados (mais de 3 bilhões de palavras) que pode ser ajustado para resultados de última geração em muitas tarefas de processamento de linguagem natural (NLP). Desde o lançamento em 2018 pelos pesquisadores do Google, ele transformou o campo da pesquisa em PNL e passou a fazer parte de melhorias significativas na [Pesquisa do Google](https://www.blog.google/products/search/search-language- compreensão-bert).

Para atender aos seus requisitos de negócios de alcançar maior precisão em um pequeno conjunto de dados (exemplos de treinamento de 20 mil), você usará uma técnica chamada aprendizado de transferência para combinar um codificador BERT pré-treinado e camadas de classificação para ajustar um novo modelo de melhor desempenho para classificação de sentimento binário .

Para este laboratório, você usará um modelo BERT menor que troca alguma precisão por tempos de treinamento mais rápidos.

Os modelos Small BERT são instâncias da arquitetura BERT original com um número menor L de camadas (ou seja, blocos residuais) combinados com um tamanho oculto H menor e um número menor A correspondente de cabeças de atenção, conforme publicado por

Iulia Turc, Ming-Wei Chang, Kenton Lee, Kristina Toutanova: ["Alunos bem lidos aprendem melhor: sobre a importância dos modelos compactos de pré-treinamento"](https://arxiv.org/abs/1908.08962), 2019.

Eles têm a mesma arquitetura geral, mas menos e/ou blocos Transformer menores, o que permite explorar compensações entre velocidade, tamanho e qualidade.

Os seguintes modelos de pré-processamento e codificador no formato TensorFlow 2 SavedModel usam a implementação do BERT do [repositório Github de modelos do TensorFlow](https://github.com/tensorflow/models/tree/master/official/nlp/bert) com o pesos treinados divulgados pelos autores do Small BERT.

In [ ]:
HPARAMS.update({
    # TF Hub BERT modules.
    "tfhub-bert-preprocessor": "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    "tfhub-bert-encoder": "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2",
})

As entradas de texto precisam ser transformadas em ids de token numéricos e organizadas em vários tensores antes de serem inseridas no BERT. O TensorFlow Hub fornece um modelo de pré-processamento correspondente para cada um dos modelos BERT discutidos acima, que implementa essa transformação usando operações TF da biblioteca TF.text. Como esse pré-processador de texto é um modelo do TensorFlow, ele pode ser incluído diretamente no seu modelo.

Para ajuste fino, você usará o mesmo otimizador com o qual o BERT foi originalmente treinado: os "Momentos Adaptativos" (Adam). Esse otimizador minimiza a perda de previsão e faz a regularização por decaimento de peso (sem usar momentos), que também é conhecido como [AdamW](https://arxiv.org/abs/1711.05101).

Para a taxa de aprendizado `inicial-learning-rate`, você usará o mesmo cronograma do pré-treinamento BERT: decaimento linear de uma taxa de aprendizado inicial nocional, prefixada com uma fase de aquecimento linear nos primeiros 10% das etapas de treinamento ` n_warmup_steps`. De acordo com o documento BERT, a taxa de aprendizado inicial é menor para ajuste fino.

In [ ]:
HPARAMS.update({
    # Hiperparâmetros de treinamento do modelo para ajuste fino e regularização.
    "epochs": 3,
    "initial-learning-rate": 3e-5,
    "dropout": 0.1 
})

In [ ]:
epochs = HPARAMS['epochs']
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
n_train_steps = steps_per_epoch * epochs
n_warmup_steps = int(0.1 * n_train_steps)    

OPTIMIZER = optimization.create_optimizer(init_lr=HPARAMS['initial-learning-rate'],
                                          num_train_steps=n_train_steps,
                                          num_warmup_steps=n_warmup_steps,
                                          optimizer_type='adamw')

### Crie e compile um classificador de sentimento do TensorFlow BERT

Em seguida, você definirá e compilará seu modelo montando componentes pré-construídos do TF-Hub e camadas tf.keras.

In [ ]:
def build_text_classifier(hparams, optimizer):
    """Define and compile a TensorFlow BERT sentiment classifier.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      model(tf.keras.Model): A compiled TensorFlow model.
    """
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    # TODO: Adicione um hub.KerasLayer para pré-processamento de texto BERT usando o dict hparams.
    # Nomeie a camada como 'preprocessing' e armazene na variável preprocessor.
    
    encoder_inputs = preprocessor(text_input)
    # TODO: Adicione um hub.KerasLayer treinável para codificação de texto BERT usando o dict hparams.
    # Nomeie a camada 'BERT_encoder' e armazene na variável encoder.

    outputs = encoder(encoder_inputs)
    # Para o ajuste fino, você usará o array `pooled_output` que representa
    # cada sequência de entrada como um todo. A forma é [batch_size, H].
    # Você pode pensar nisso como uma incorporação para toda a crítica do filme.
    classifier = outputs['pooled_output']
    # Adicione dropout para evitar overfitting durante o ajuste fino do modelo.
    classifier = tf.keras.layers.Dropout(hparams['dropout'], name='dropout')(classifier)
    classifier = tf.keras.layers.Dense(1, activation=None, name='classifier')(classifier)
    model = tf.keras.Model(text_input, classifier, name='bert-sentiment-classifier')
    
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()    
    
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics)    
    
    return model

In [ ]:
model = build_text_classifier(HPARAMS, OPTIMIZER)

In [ ]:
# Visualize seu classificador de sentimento BERT ajustado.
tf.keras.utils.plot_model(model)

In [ ]:
TEST_REVIEW = ['this is such an amazing movie!']

In [ ]:
BERT_RAW_RESULT = model(tf.constant(TEST_REVIEW))
print(BERT_RAW_RESULT)

### Train and evaluate your BERT sentiment classifier

In [ ]:
HPARAMS.update({
    # TODO: Salve seu classificador de sentimento BERT localmente.
    # Dica: Salve-o em './bert-sentiment-classifier-local'. Observe o nome da chave em model.save().
    
})

**Note:** training your model locally will take about 8-10 minutes.

In [ ]:
def train_evaluate(hparams):
    """Train and evaluate TensorFlow BERT sentiment classifier.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      history(tf.keras.callbacks.History): Keras callback that records training event history.
    """
    # dataset_dir = download_data(data_url, local_data_dir)
    raw_train_ds, raw_val_ds, raw_test_ds = load_datasets(DATASET_DIR, hparams)
    
    train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)     
    
    epochs = hparams['epochs']
    steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
    n_train_steps = steps_per_epoch * epochs
    n_warmup_steps = int(0.1 * n_train_steps)    
    
    optimizer = optimization.create_optimizer(init_lr=hparams['initial-learning-rate'],
                                              num_train_steps=n_train_steps,
                                              num_warmup_steps=n_warmup_steps,
                                              optimizer_type='adamw')    
    
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = build_text_classifier(hparams=hparams, optimizer=optimizer)
    
    logging.info(model.summary())
        
    history = model.fit(x=train_ds,
                        validation_data=val_ds,
                        epochs=epochs)  
    
    logging.info("Test accuracy: %s", model.evaluate(test_ds))

    # Export Keras model in TensorFlow SavedModel format.
    model.save(hparams['model-dir'])
    
    return history

Baseado no objeto `History` retornado por `model.fit()`. Você pode plotar a perda de treinamento e validação para comparação, bem como a precisão de treinamento e validação:

In [ ]:
history = train_evaluate(HPARAMS)

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right');

Nesse gráfico, as linhas vermelhas representam a perda e a precisão do treinamento e as linhas azuis são a perda e a precisão da validação. Com base nos gráficos acima, você deve ver a precisão do modelo em torno de 78-80%, o que excede sua meta de requisitos de negócios de mais de 75% de precisão.

## Conteinerize seu código de modelo

Agora que você treinou e avaliou seu modelo localmente em um Vertex Notebook como parte de um fluxo de trabalho de experimentação, sua próxima etapa é treinar e implantar seu modelo na plataforma Vertex AI do Google Cloud.

Para treinar seu classificador BERT no Google Cloud, você empacotará seus scripts de treinamento Python e escreverá um Dockerfile que contém instruções sobre o código do modelo de ML, dependências e instruções de execução. Você criará seu contêiner personalizado com o Cloud Build, cujas instruções são especificadas em `cloudbuild.yaml` e publicará seu contêiner em seu Artifact Registry. Esse fluxo de trabalho oferece a oportunidade de usar o mesmo contêiner para executar como parte de um fluxo de trabalho [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) portátil e escalável.


Você passará pela criação da seguinte estrutura de projeto para o código do modo ML:
```
|--/bert-sentiment-classifier
   |--/trainer
      |--__init__.py
      |--model.py
      |--task.py
   |--Dockerfile
   |--cloudbuild.yaml
   |--requirements.txt
```

### 1. Escreva um script de treinamento `model.py`

Primeiro, você organizará seu código de treinamento do modelo TensorFlow local acima em um script de treinamento.

In [ ]:
MODEL_DIR = "bert-sentiment-classifier"

In [ ]:
%%writefile {MODEL_DIR}/trainer/model.py
import os
import shutil
import logging

import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
from official.nlp import optimization

DATA_URL = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
LOCAL_DATA_DIR = './tmp/data'
AUTOTUNE = tf.data.AUTOTUNE


def download_data(data_url, local_data_dir):
    """Download dataset.
    Args:
      data_url(str): Source data URL path.
      local_data_dir(str): Local data download directory path.
    Returns:
      dataset_dir(str): Local unpacked data directory path.
    """
    if not os.path.exists(local_data_dir):
        os.makedirs(local_data_dir)
    
    dataset = tf.keras.utils.get_file(
      fname='aclImdb_v1.tar.gz',
      origin=data_url,
      untar=True,
      cache_dir=local_data_dir,
      cache_subdir="")
    
    dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
    
    train_dir = os.path.join(dataset_dir, 'train')
    
    # Remove unused folders to make it easier to load the data.
    remove_dir = os.path.join(train_dir, 'unsup')
    shutil.rmtree(remove_dir)
    
    return dataset_dir


def load_datasets(dataset_dir, hparams):
    """Load pre-split tf.datasets.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      raw_train_ds(tf.dataset): Train split dataset (20k examples).
      raw_val_ds(tf.dataset): Validation split dataset (5k examples).
      raw_test_ds(tf.dataset): Test split dataset (25k examples).
    """    

    raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'train'),
        batch_size=hparams['batch-size'],
        validation_split=0.2,
        subset='training',
        seed=hparams['seed'])    

    raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'train'),
        batch_size=hparams['batch-size'],
        validation_split=0.2,
        subset='validation',
        seed=hparams['seed'])

    raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'test'),
        batch_size=hparams['batch-size'])
    
    return raw_train_ds, raw_val_ds, raw_test_ds


def build_text_classifier(hparams, optimizer):
    """Define and compile a TensorFlow BERT sentiment classifier.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      model(tf.keras.Model): A compiled TensorFlow model.
    """
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    # TODO: Adicione um hub.KerasLayer para pré-processamento de texto BERT usando o dict hparams.
    # Nomeie a camada como 'preprocessing' e armazene na variável preprocessor.
    preprocessor = hub.KerasLayer(hparams['tfhub-bert-preprocessor'], name='preprocessing')
    encoder_inputs = preprocessor(text_input)
    # TODO: Adicione um hub.KerasLayer treinável para codificação de texto BERT usando o dict hparams.
    # Nomeie a camada 'BERT_encoder' e armazene na variável encoder.
    encoder = hub.KerasLayer(hparams['tfhub-bert-encoder'], trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    # Para o ajuste fino, você usará o array `pooled_output` que representa
    # cada sequência de entrada como um todo. A forma é [batch_size, H].
    # Você pode pensar nisso como uma incorporação para toda a crítica do filme.
    classifier = outputs['pooled_output']
    # Adicione dropout para evitar overfitting durante o ajuste fino do modelo.
    classifier = tf.keras.layers.Dropout(hparams['dropout'], name='dropout')(classifier)
    classifier = tf.keras.layers.Dense(1, activation=None, name='classifier')(classifier)
    model = tf.keras.Model(text_input, classifier, name='bert-sentiment-classifier')
    
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()    
    
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics)    
    
    return model


def train_evaluate(hparams):
    """Train and evaluate TensorFlow BERT sentiment classifier.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      history(tf.keras.callbacks.History): Keras callback that records training event history.
    """
    dataset_dir = download_data(data_url=DATA_URL, 
                                local_data_dir=LOCAL_DATA_DIR)
    
    raw_train_ds, raw_val_ds, raw_test_ds = load_datasets(dataset_dir=dataset_dir,
                                                          hparams=hparams)
    
    train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)     
    
    epochs = hparams['epochs']
    steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
    n_train_steps = steps_per_epoch * epochs
    n_warmup_steps = int(0.1 * n_train_steps)    
    
    optimizer = optimization.create_optimizer(init_lr=hparams['initial-learning-rate'],
                                              num_train_steps=n_train_steps,
                                              num_warmup_steps=n_warmup_steps,
                                              optimizer_type='adamw')    
    
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = build_text_classifier(hparams=hparams, optimizer=optimizer)
        logging.info(model.summary())
        
    history = model.fit(x=train_ds,
                        validation_data=val_ds,
                        epochs=epochs)  
    
    logging.info("Test accuracy: %s", model.evaluate(test_ds))

    # Exporte o modelo Keras no formato TensorFlow SavedModel.
    model.save(hparams['model-dir'])
    
    return history

### 2. Escreva um arquivo `task.py` como um ponto de entrada para seu contêiner de modelo personalizado

In [ ]:
%%writefile {MODEL_DIR}/trainer/task.py

import os
import argparse

from trainer import model

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    # Argumentos de treinamento de contêiner personalizado de vértice. Eles são definidos pelo Vertex AI durante o treinamento, mas também podem ser substituídos.
    parser.add_argument('--model-dir', dest='model-dir',
                        default=os.environ['AIP_MODEL_DIR'], type=str, help='GCS URI for saving model artifacts.')

    # Argumentos de treinamento do modelo.
    parser.add_argument('--tfhub-bert-preprocessor', dest='tfhub-bert-preprocessor', 
                        default='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3', type=str, help='TF-Hub URL.')
    parser.add_argument('--tfhub-bert-encoder', dest='tfhub-bert-encoder', 
                        default='https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2', type=str, help='TF-Hub URL.')    
    parser.add_argument('--initial-learning-rate', dest='initial-learning-rate', default=3e-5, type=float, help='Learning rate for optimizer.')
    parser.add_argument('--epochs', dest='epochs', default=3, type=int, help='Training iterations.')    
    parser.add_argument('--batch-size', dest='batch-size', default=32, type=int, help='Number of examples during each training iteration.')    
    parser.add_argument('--dropout', dest='dropout', default=0.1, type=float, help='Float percentage of DNN nodes [0,1] to drop for regularization.')    
    parser.add_argument('--seed', dest='seed', default=42, type=int, help='Random number generator seed to prevent overlap between train and val sets.')
    
    args = parser.parse_args()
    hparams = args.__dict__

    model.train_evaluate(hparams)

### 3. Escreva um `Dockerfile` para seu contêiner de modelo personalizado

Terceiro, você escreverá um `Dockerfile` que contém instruções para empacotar o código do seu modelo em `bert-sentiment-classifier`, bem como especificar as dependências do código do seu modelo necessárias para execução em um contêiner do Docker.

In [ ]:
%%writefile {MODEL_DIR}/Dockerfile
# Especifica a imagem de base e a marca.
# https://cloud.google.com/vertex-ai/docs/training/pre-built-containers
FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-6:latest

# Define o diretório de trabalho do contêiner.
WORKDIR /root

# Copia o requirements.txt no contêiner para reduzir as chamadas de rede.
COPY requirements.txt .

# Instala pacotes adicionais.
RUN pip3 install -U -r requirements.txt

# b/203105209 Remove o arquivo desnecessário da imagem de CPU TF2.5 para treinamento de python_module CustomJob.
# Será removido nas imagens públicas de Vertex subsequentes.
RUN rm -rf /var/sitecustomize/sitecustomize.py

# Copia o código do treinamento para a imagem do docker.
COPY . /trainer

# Define o diretório de trabalho do contêiner.
WORKDIR /trainer

# Configura o ponto de entrada para invocar o treinamento.
ENTRYPOINT ["python", "-m", "trainer.task"]

### 4. Escreva um arquivo `requirements.txt` para especificar dependências de código ML adicionais

Essas são dependências adicionais para o código do seu modelo não incluídas nas imagens do TensorFlow do Vertex pré-criadas, como TF-Hub, otimizador TensorFlow AdamW e texto do TensorFlow necessários para importar e trabalhar com modelos TensorFlow BERT pré-treinados.

In [ ]:
%%writefile {MODEL_DIR}/requirements.txt
tf-models-official==2.6.0
tensorflow-text==2.6.0
tensorflow-hub==0.12.0

## Use o Cloud Build para criar e enviar seu contêiner de modelo ao Google Cloud Artifact Registry

Em seguida, você usará o [Cloud Build](https://cloud.google.com/build) para criar e fazer upload do seu contêiner de modelo personalizado do TensorFlow para o [Google Cloud Artifact Registry](https://cloud.google.com/artifact-registro).

O Cloud Build traz reutilização e automação para sua experimentação de ML, permitindo que você crie, teste e implante de forma confiável o código do modelo de ML como parte de um fluxo de trabalho de CI/CD. O Artifact Registry fornece um repositório centralizado para você armazenar, gerenciar e proteger suas imagens de contêiner de ML. Isso permitirá que você compartilhe com segurança seu trabalho de ML com outras pessoas e reproduza os resultados do experimento.

**Observação**: a etapa inicial de compilação e envio levará cerca de 16 minutos, mas o Cloud Build pode aproveitar o armazenamento em cache para compilações subsequentes mais rápidas.

### 1. Create Artifact Registry for custom container images

In [ ]:
ARTIFACT_REGISTRY="bert-sentiment-classifier"

In [ ]:
# TODO: crie um Docker Artifact Registry usando a CLI gcloud. Observe os sinalizadores de formato e local de repositório necessários.
# Link da documentação: https://cloud.google.com/sdk/gcloud/reference/artifacts/repositories/create


### 2. Crie instruções `cloudbuild.yaml`

In [ ]:
IMAGE_NAME="bert-sentiment-classifier"
IMAGE_TAG="latest"
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REGISTRY}/{IMAGE_NAME}:{IMAGE_TAG}"

In [ ]:
cloudbuild_yaml = f"""steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '{IMAGE_URI}', '.' ]
images: 
- '{IMAGE_URI}'"""

with open(f"{MODEL_DIR}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

### 3. Crie e envie sua imagem de contêiner para o Artifact Registry usando o Cloud Build

**Observação:** seu contêiner de modelo personalizado levará cerca de 16 minutos inicialmente para ser criado e enviado ao seu Artifact Registry. O Artifact Registry é capaz de aproveitar o armazenamento em cache para que as compilações subsequentes demorem cerca de 4 minutos.

In [ ]:
# TODO: use o Cloud Build para criar e enviar seu contêiner de modelo personalizado ao Artifact Registry.
# Link da documentação: https://cloud.google.com/sdk/gcloud/reference/builds/submit
# Dica: certifique-se de que o sinalizador de configuração esteja apontado para {MODEL_DIR}/cloudbuild.yaml definido acima e inclua seu diretório de modelo.



## Defina um pipeline usando o SDK do KFP V2

Para atender aos seus requisitos de negócios e colocar seu modelo de melhor desempenho em produção para agregar valor mais rapidamente, você definirá um pipeline usando o [**Kubeflow Pipelines (KFP) V2 SDK**](https://www.kubeflow.org/docs/components/pipelines/sdk/v2/v2-compatibility) para orquestrar o treinamento e a implantação do seu modelo no [**Vertex Pipelines**](https://cloud.google.com/vertex-ai/docs/pipelines) abaixo de.

In [ ]:
import datetime
# google_cloud_pipeline_components includes pre-built KFP components for interfacing with Vertex AI services.
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import dsl

In [ ]:
TIMESTAMP=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
DISPLAY_NAME = "bert-sentiment-{}".format(TIMESTAMP)
GCS_BASE_OUTPUT_DIR= f"{GCS_BUCKET}/{MODEL_DIR}-{TIMESTAMP}"

USER = ""  # TODO: mude isso para o seu nome.
PIPELINE_ROOT = "{}/pipeline_root/{}".format(GCS_BUCKET, USER)

print(f"Model display name: {DISPLAY_NAME}")
print(f"GCS dir for model training artifacts: {GCS_BASE_OUTPUT_DIR}")
print(f"GCS dir for pipeline artifacts: {PIPELINE_ROOT}")

In [ ]:
# Contêiner de serviço de modelo Vertex pré-construído para implantação.
# https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
SERVING_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest"

O pipeline consiste em três componentes:

* `CustomContainerTrainingJobRunOp` [(documentação)](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.CustomContainerTrainingJobRunOp): treina seu contêiner de modelo personalizado usando o Vertex Training. Isso é o mesmo que configurar um trabalho de treinamento de contêiner personalizado Vertex usando o Vertex Python SDK que você abordou no laboratório Vertex AI: Qwik Start.

* `EndpointCreateOp` [(documentação)](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.EndpointCreateOp): cria um recurso do Google Cloud Vertex Endpoint que mapeia recursos de máquina física com seu modelo para permitir que ele veicule previsões on-line. As previsões online têm requisitos de baixa latência; fornecer recursos para o modelo com antecedência reduz a latência.

* `ModelDeployOp`[(documentação)](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-0.2.0/google_cloud_pipeline_components.aiplatform.html#google_cloud_pipeline_components.aiplatform.ModelDeployOp): implanta seu modelo em um Vertex Prediction Endpoint para previsões online.

In [ ]:
@dsl.pipeline(name="bert-sentiment-classification", pipeline_root=PIPELINE_ROOT)
def pipeline(
    project: str = PROJECT_ID,
    location: str = REGION,
    staging_bucket: str = GCS_BUCKET,
    display_name: str = DISPLAY_NAME,    
    container_uri: str = IMAGE_URI,
    model_serving_container_image_uri: str = SERVING_IMAGE_URI,    
    base_output_dir: str = GCS_BASE_OUTPUT_DIR,
):
    
    #TODO: adicione e configure o componente KFP CustomContainerTrainingJobRunOp pré-criado usando
    # os argumentos restantes no construtor do pipeline.
    # Dica: Consulte o link de documentação do componente acima, se necessário também.
    model_train_evaluate_op = gcc_aip.CustomContainerTrainingJobRunOp(
        # Parâmetros de autenticação Vertex AI Python SDK.      
        project=project,
        location=location,
        staging_bucket=staging_bucket,
        # WorkerPool arguments.
        replica_count=1,
        machine_type="c2-standard-4",
        # TODO: preencha os argumentos restantes do construtor de pipeline.

    )    
    
    # Crie um recurso Vertex Endpoint em paralelo com o treinamento do modelo.
    endpoint_create_op = gcc_aip.EndpointCreateOp(
        # Parâmetros de autenticação Vertex AI Python SDK.
        project=project,
        location=location,
        display_name=display_name
    
    )   
    
    # Implante seu modelo no recurso Endpoint criado para previsões online.
    model_deploy_op = gcc_aip.ModelDeployOp(
        # Link para o componente de treinamento do modelo por meio do artefato do modelo de saída.
        model=model_train_evaluate_op.outputs["model"],
        # Link para o Endpoint criado.
        endpoint=endpoint_create_op.outputs["endpoint"],
        # Defina o roteamento de solicitação de previsão. {"0": 100} indica 100% do tráfego
        # para o ID do modelo atual que está sendo implantado.
        traffic_split={"0": 100},
        # Argumentos do WorkerPool.
        dedicated_resources_machine_type="n1-standard-4",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=2
    )

## Compilar o pipeline

In [ ]:
from kfp.v2 import compiler

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="bert-sentiment-classification.json"
)

## Execute o pipeline no Vertex Pipelines

O `PipelineJob` é configurado abaixo e acionado através do método `run()`.

**Observação:** essa execução de pipeline levará cerca de 30 a 40 minutos para treinar e implantar seu modelo. Acompanhe a execução usando o URL da saída do trabalho abaixo.

In [ ]:
vertex_pipelines_job = vertexai.pipeline_jobs.PipelineJob(
    display_name="bert-sentiment-classification",
    template_path="bert-sentiment-classification.json",
    parameter_values={
        "project": PROJECT_ID,
        "location": REGION,
        "staging_bucket": GCS_BUCKET,
        "display_name": DISPLAY_NAME,        
        "container_uri": IMAGE_URI,
        "model_serving_container_image_uri": SERVING_IMAGE_URI,        
        "base_output_dir": GCS_BASE_OUTPUT_DIR},
    enable_caching=True,
)

In [ ]:
vertex_pipelines_job.run()

## Modelo implantado de consulta no Vertex Endpoint para previsões online

Por fim, você recuperará o `endpoint` implantado pelo pipeline e o usará para consultar seu modelo para previsões online.

Configure a função `Endpoint()` abaixo com os seguintes parâmetros:

* `endpoint_name`: um nome de recurso de terminal ou ID de terminal totalmente qualificado. Exemplo: "projects/123/locations/us-central1/endpoints/456" ou "456" quando o projeto e o local são inicializados ou passados.
* `project_id`: projeto do GCP.
* `local`: região do GCP.

Chame `predict()` para retornar uma previsão para uma revisão de teste.

In [ ]:
# Rrecupere o nome do Endpoint implantado de seu pipeline.
ENDPOINT_NAME = vertexai.Endpoint.list()[0].name

In [ ]:
#TODO: Gere previsões online usando seu Vertex Endpoint.

endpoint = vertexai.Endpoint(
)

In [ ]:
#TODO: escreva uma resenha de filme para testar seu modelo, por exemplo "The Dark Knight is the best Batman movie!"
test_review = ""

In [ ]:
# TODO: use seu endpoint para retornar a previsão para seu test_review.
prediction =

In [ ]:
print(prediction)

In [ ]:
# Use uma função sigmoid para compactar a saída do seu modelo entre 0 e 1. Para classificação binária, normalmente é aplicado um limite de 0,5
# portanto, se a saída for >= 0,5, o sentimento previsto será "Positivo" e < 0,5 será uma previsão "Negativa".
print(tf.sigmoid(prediction.predictions[0]))

## Próximos Passos

Parabéns! Você percorreu um fluxo de trabalho completo de experimentação, conteinerização e MLOps no Vertex AI. Primeiro, você construiu, treinou e avaliou um modelo de classificador de sentimento BERT em um Vertex Notebook. Em seguida, você empacotou o código do modelo em um contêiner do Docker para treinar no Vertex AI do Google Cloud. Por fim, você definiu e executou um Kubeflow Pipeline em Vertex Pipelines que treinou e implantou seu contêiner de modelo em um Vertex Endpoint que você consultou para previsões online.

## License

In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.